## initialize

In [1]:
%cd ..

/Users/ales/dev/repos/ai-audio-books


/Users/ales/dev/python-venvs/ai-audio-books/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os

from pprint import pprint

import dotenv
import pandas as pd
from httpx import Timeout
from pydantic import BaseModel
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_openai import ChatOpenAI
from langchain_community.callbacks import get_openai_callback

from IPython.display import Audio

import data.samples_to_split as samples

from src.lc_callbacks import LCMessageLoggerAsync
from src.schemas import AudioOutputFormat, TTSParams, TTSTimestampsResponse, TTSTimestampsAlignemnt
from src.text_split_chain import create_split_text_chain
from src import tts
from src.utils import GPTModels

In [4]:
dotenv.load_dotenv()

True

In [6]:
text = samples.GATSBY_2
# text = """\
# Margaret: hello, how are you Tom?
# Tom: nice, thanks. And you?
# """

## split text into character phrases

In [6]:
chain = create_split_text_chain(llm_model=GPTModels.GPT_4o)
# chain = create_split_text_chain(llm_model=GPTModels.GPT_4_TURBO_2024_04_09)
with get_openai_callback() as cb:
    res = chain.invoke({"text": text}, config={"callbacks": [LCMessageLoggerAsync()]})

2024-11-01 20:39:39,640 [INFO] audio-books (lc_callbacks.py): call to gpt-4o with 2 messages:
{'role': 'system', 'content': 'you are provided with the book sample.\nplease rewrite it and insert xml tags indicating character to whom current phrase belongs.\nfor example: <narrator>I looked at her</narrator><Jill>What are you looking at?</Jill>\n\nNotes:\n- sometimes narrator is one of characters taking part in the action.\nin this case use narrator\'s name (if available) instead of "narrator"\n- if it\'s impossible to identify character name from the text provided, use codes "c1", "c2", etc,\nwhere "c" prefix means character and number is used to enumerate unknown characters\n- all quotes of direct speech must be attributed to characters, for example:\n<Tom>“She’s a nice girl,”</Tom><narrator>said Tom after a moment.</narrator>\nmind that sometimes narrator could also be a character.\n- use ALL available context to determine the character.\nsometimes the character name becomes clear from

In [7]:
res.characters

['c1', 'narrator', 'Jordan', 'Tom', 'Daisy']

In [8]:
print(res.text_annotated)

<narrator>Inside, the crimson room bloomed with light. Tom and Miss Baker sat at either end of the long couch and she read aloud to him from the Saturday Evening Post—the words, murmurous and uninflected, running together in a soothing tune. The lamplight, bright on his boots and dull on the autumn-leaf yellow of her hair, glinted along the paper as she turned a page with a flutter of slender muscles in her arms.</narrator>

<narrator>When we came in she held us silent for a moment with a lifted hand.</narrator>

<Jordan>“To be continued,”</Jordan> <narrator>she said, tossing the magazine on the table,</narrator> <Jordan>“in our very next issue.”</Jordan>

<narrator>Her body asserted itself with a restless movement of her knee, and she stood up.</narrator>

<Jordan>“Ten o’clock,”</Jordan> <narrator>she remarked, apparently finding the time on the ceiling.</narrator> <Jordan>“Time for this good girl to go to bed.”</Jordan>

<Daisy>“Jordan’s going to play in the tournament tomorrow,”</Da

In [9]:
res.phrases

[CharacterPhrase(character='narrator', text='Inside, the crimson room bloomed with light. Tom and Miss Baker sat at either end of the long couch and she read aloud to him from the Saturday Evening Post—the words, murmurous and uninflected, running together in a soothing tune. The lamplight, bright on his boots and dull on the autumn-leaf yellow of her hair, glinted along the paper as she turned a page with a flutter of slender muscles in her arms.'),
 CharacterPhrase(character='narrator', text='When we came in she held us silent for a moment with a lifted hand.'),
 CharacterPhrase(character='Jordan', text='“To be continued,”'),
 CharacterPhrase(character='narrator', text='she said, tossing the magazine on the table,'),
 CharacterPhrase(character='Jordan', text='“in our very next issue.”'),
 CharacterPhrase(character='narrator', text='Her body asserted itself with a restless movement of her knee, and she stood up.'),
 CharacterPhrase(character='Jordan', text='“Ten o’clock,”'),
 Characte

In [10]:
print(res.text_raw)

Inside, the crimson room bloomed with light. Tom and Miss Baker sat at
either end of the long couch and she read aloud to him from the
Saturday Evening Post—the words, murmurous and uninflected, running
together in a soothing tune. The lamplight, bright on his boots and
dull on the autumn-leaf yellow of her hair, glinted along the paper as
she turned a page with a flutter of slender muscles in her arms.

When we came in she held us silent for a moment with a lifted hand.

“To be continued,” she said, tossing the magazine on the table, “in
our very next issue.”

Her body asserted itself with a restless movement of her knee, and she
stood up.

“Ten o’clock,” she remarked, apparently finding the time on the
ceiling. “Time for this good girl to go to bed.”

“Jordan’s going to play in the tournament tomorrow,” explained Daisy,
“over at Westchester.”

“Oh—you’re Jordan Baker.”

I knew now why her face was familiar—its pleasing contemptuous
expression had looked out at me from many rotogravur

In [11]:
print(res.to_pretty_text())

characters: ['c1', 'narrator', 'Jordan', 'Tom', 'Daisy']
--------------------
[narrator] Inside, the crimson room bloomed with light. Tom and Miss Baker sat at either end of the long couch and she read aloud to him from the Saturday Evening Post—the words, murmurous and uninflected, running together in a soothing tune. The lamplight, bright on his boots and dull on the autumn-leaf yellow of her hair, glinted along the paper as she turned a page with a flutter of slender muscles in her arms.
[narrator] When we came in she held us silent for a moment with a lifted hand.
[Jordan] “To be continued,”
[narrator] she said, tossing the magazine on the table,
[Jordan] “in our very next issue.”
[narrator] Her body asserted itself with a restless movement of her knee, and she stood up.
[Jordan] “Ten o’clock,”
[narrator] she remarked, apparently finding the time on the ceiling.
[Jordan] “Time for this good girl to go to bed.”
[Daisy] “Jordan’s going to play in the tournament tomorrow,”
[narrator] 

In [12]:
print(f'LLM usage:\n\n{cb}')

LLM usage:

Tokens Used: 1817
	Prompt Tokens: 877
	Completion Tokens: 940
Successful Requests: 1
Total Cost (USD): $0.0115925


## map characters to voices

In [13]:
from src.select_voice_chain import VoiceSelector

In [14]:
vs = VoiceSelector()

2024-11-01 20:40:27,198 [INFO] audio-books (select_voice_chain.py): reading voice data from: "data/11labs_available_tts_voices.reviewed.csv"
2024-11-01 20:40:27,208 [INFO] audio-books (select_voice_chain.py): df.shape=(34, 15)
2024-11-01 20:40:27,211 [INFO] audio-books (select_voice_chain.py): filtering df by "manual_quality_review" column
2024-11-01 20:40:27,216 [INFO] audio-books (select_voice_chain.py): df.shape after filtering voices: (25, 15)


In [15]:
chain = vs.create_voice_mapping_chain(llm_model=GPTModels.GPT_4o)

In [16]:
chain

RunnableAssign(mapper={
  charater_props: ChatPromptTemplate(input_variables=['characters', 'text'], input_types={}, partial_variables={'available_genders': '"male", "female"', 'available_age_groups': '"young", "middle_aged", "old"', 'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"CharacterProperties": {"properties": {"gender": {"title": "Gender", "type": "string"}, "age_group": {"title": "Age Group", "type": "string"}}, "required": ["gender", "age_group"], "title": "CharacterProperties", "type": "object"}}, "properties": {"character2props": {"addi

In [17]:
res2 = chain.invoke(
    {"text": res.text_annotated, "characters": res.characters},
    config={"callbacks": [LCMessageLoggerAsync()]},
)

2024-11-01 20:40:29,935 [INFO] audio-books (lc_callbacks.py): call to gpt-4o with 2 messages:
{'role': 'system', 'content': 'You are a helpful assistant proficient in literature and psychology.\nOur goal is to create an audio book from the given text.\nFor that we need to hire voice actors.\nPlease help us to find the right actor for each character present in the text.\n\nYou are provided with the text split by the characters\nto whom text parts belong to.\n\nYour task is to assign available properties to each character provided.\nList of available properties:\n- gender: "male", "female"\n- age_group: "young", "middle_aged", "old"\n\nNOTES:\n- assign EXACTLY ONE property value for each property\n- select properties values ONLY from the list of AVAILABLE property values\n- fill properties for ALL characters from the list provided\n- DO NOT include any characters absent in the list provided\n\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs

In [18]:
res2

SelectVoiceChainOutput(character2props={'c1': CharacterPropertiesNullable(gender='male', age_group='young'), 'narrator': CharacterPropertiesNullable(gender='male', age_group='young'), 'Jordan': CharacterPropertiesNullable(gender='female', age_group='young'), 'Tom': CharacterPropertiesNullable(gender='male', age_group='middle_aged'), 'Daisy': CharacterPropertiesNullable(gender='female', age_group='young')}, character2voice={'narrator': '1RVpBInY9YUYMLSUQReV', 'c1': 'xSI29a9HAKdsWv3idXSN', 'Jordan': 'rfkTsdZrVWEVhDycUYn9', 'Daisy': 'U0gWepDCHKmmexbBxeSX', 'Tom': 'cjVigY5qzO86Huf0OWal'})

In [19]:
character2voice = res2.character2voice
character2voice

{'narrator': '1RVpBInY9YUYMLSUQReV',
 'c1': 'xSI29a9HAKdsWv3idXSN',
 'Jordan': 'rfkTsdZrVWEVhDycUYn9',
 'Daisy': 'U0gWepDCHKmmexbBxeSX',
 'Tom': 'cjVigY5qzO86Huf0OWal'}

## generate audio

In [20]:
res.phrases

[CharacterPhrase(character='narrator', text='Inside, the crimson room bloomed with light. Tom and Miss Baker sat at either end of the long couch and she read aloud to him from the Saturday Evening Post—the words, murmurous and uninflected, running together in a soothing tune. The lamplight, bright on his boots and dull on the autumn-leaf yellow of her hair, glinted along the paper as she turned a page with a flutter of slender muscles in her arms.'),
 CharacterPhrase(character='narrator', text='When we came in she held us silent for a moment with a lifted hand.'),
 CharacterPhrase(character='Jordan', text='“To be continued,”'),
 CharacterPhrase(character='narrator', text='she said, tossing the magazine on the table,'),
 CharacterPhrase(character='Jordan', text='“in our very next issue.”'),
 CharacterPhrase(character='narrator', text='Her body asserted itself with a restless movement of her knee, and she stood up.'),
 CharacterPhrase(character='Jordan', text='“Ten o’clock,”'),
 Characte

In [21]:
tts_responses = []

for phrase in res.phrases:
    voice_id = character2voice[phrase.character]
    tts_params = params = TTSParams(voice_id=voice_id, text=phrase.text)
    response = await tts.tts_w_timestamps(params)
    tts_responses.append(response)

2024-11-01 20:41:03,150 [INFO] audio-books (tts.py): request to 11labs TTS endpoint with params {'voice_id': '1RVpBInY9YUYMLSUQReV', 'output_format': <AudioOutputFormat.MP3_44100_192: 'mp3_44100_192'>} for the following text: "Inside, the crimson room bloomed with light. Tom and Miss Baker sat at either end of the long couch and she read aloud to him from the Saturday Evening Post—the words, murmurous and uninflected, running together in a soothing tune. The lamplight, bright on his boots and dull on the autumn-leaf yellow of her hair, glinted along the paper as she turned a page with a flutter of slender muscles in her arms."
2024-11-01 20:41:08,634 [INFO] httpx (_client.py): HTTP Request: POST https://api.elevenlabs.io/v1/text-to-speech/1RVpBInY9YUYMLSUQReV/with-timestamps?output_format=mp3_44100_192 "HTTP/1.1 200 OK"
2024-11-01 20:41:08,759 [INFO] audio-books (tts.py): request to 11labs TTS endpoint with params {'voice_id': '1RVpBInY9YUYMLSUQReV', 'output_format': <AudioOutputFormat

In [22]:
alignments = [response.alignment for response in tts_responses]

In [23]:
char2time = TTSTimestampsAlignemnt.combine_alignments(alignments=alignments)

In [67]:
char2time.to_dataframe()

,char,start,end
0,I,0.000,0.232
1,n,0.232,0.383
2,s,0.383,0.464
3,i,0.464,0.615
4,d,0.615,0.673
...,...,...,...
2532,d,151.158,151.205
2533,,151.205,151.263
2534,m,151.263,151.309
2535,e,151.309,151.402


In [37]:
print(f'len of original text: {len(text)}')
# add 1 extra char imitating space between character phrases
joined_phrases = ' '.join([x.text for x in res.phrases])
print(f'len of joined character phrases: {len(joined_phrases)}')
print(f'len of char2time mapping: {len(char2time.characters)}')

len of original text: 2560
len of joined character phrases: 2537
len of char2time mapping: 2537


## design effects

In [9]:
from src.sound_effects_design import create_sound_effects_design_chain, SoundEffectDescription

In [10]:
chain = create_sound_effects_design_chain(llm_model=GPTModels.GPT_4o)

with get_openai_callback() as cb:
    res = chain.invoke(
        {"text": text}, config={"callbacks": [LCMessageLoggerAsync()]}
    )

2024-11-03 11:16:01,132 [INFO] audio-books (lc_callbacks.py): call to gpt-4o with 2 messages:
{'role': 'system', 'content': 'You are an expert in directing audiobooks creation.\nYour task is to design sound effects layed over the audio book.\nYou are provided with the audiobook text chunk -\ninsert XML tags describing sound effects, their place and duration.\n\nXML effect tags must have following structure:\n<effect prompt="prompt to be passed to text-to-sound-effect AI model">original line from the text</effect>\n\nAdditional requirements:\n- In the very beginning, analyze the whole text chunk provided in order to understand events and atmosphere.\n- Prompts you place inside XML tags are going to be passes to text-to-sound-effect AI model.\nThus, it\'s required to write prompts rich in details.\n- Do not generate long-running background or ambient music, crowd talking\n- Aim for episodical sound effects, highlighting atmosphere and characters\' actions.\nFor example, cracking of stair

In [11]:
# print(res.text_raw)
# print(res.text_annotated)

In [12]:
res.sound_effects_descriptions

[SoundEffectDescription(prompt='room filled with warm, inviting light', text_between_tags='crimson room bloomed with light', ix_start_llm_response=12, ix_end_llm_response=107, ix_start_orig_text=12, ix_end_orig_text=43, duration=-1),
 SoundEffectDescription(prompt='soft rustling of paper as a page is turned', text_between_tags='glinted along the paper as she turned a page with a flutter of slender muscles in her arms', ix_start_llm_response=379, ix_end_llm_response=538, ix_start_orig_text=315, ix_end_orig_text=405, duration=-1),
 SoundEffectDescription(prompt='gentle, commanding gesture', text_between_tags='lifted hand', ix_start_llm_response=596, ix_end_llm_response=660, ix_start_orig_text=463, ix_end_orig_text=474, duration=-1),
 SoundEffectDescription(prompt='magazine landing softly on a table', text_between_tags='tossing the magazine on the table', ix_start_llm_response=692, ix_end_llm_response=786, ix_start_orig_text=506, ix_end_orig_text=539, duration=-1),
 SoundEffectDescription

In [15]:
sed = res.sound_effects_descriptions[0]
print(f'{sed.duration=}')

SoundEffectDescription(prompt='room filled with warm, inviting light', text_between_tags='crimson room bloomed with light', ix_start_llm_response=12, ix_end_llm_response=107, ix_start_orig_text=12, ix_end_orig_text=43, duration=-1)

In [16]:
# TODO: move to function
ix_start, ix_end = sed.ix_start_llm_response, sed.ix_end_llm_response
time_start = char2time.get_start_time_by_char_ix(ix_start)
time_end = char2time.get_end_time_by_char_ix(ix_end)
duration = time_end - time_start
sed.duration = duration
print(f'{sed.duration=}')

# TODO: filter out sound effects with duration < X seconds (X = 1.0 ?)

In [17]:
char2time.to_dataframe().head(10)

NameError: name 'char2time' is not defined

In [65]:
for sed in res.sound_effects_descriptions:
    text_bw_tags = sed.text_between_tags
    orig_text_from_calc_indices = text[sed.ix_start_orig_text : sed.ix_end_orig_text]
    print(f'{sed.prompt=}')
    print(f'{text_bw_tags=}')
    print(f'{orig_text_from_calc_indices=}')
    print(f'{orig_text_from_calc_indices == text_bw_tags=}')
    print(f'')

sed.prompt='room filled with warm, inviting light'
text_bw_tags='crimson room bloomed with light'
orig_text_from_calc_indices='crimson room bloomed with light'
orig_text_from_calc_indices == text_bw_tags=True

sed.prompt='soft rustling of paper as a page is turned'
text_bw_tags='glinted along the paper as she turned a page with a flutter of slender muscles in her arms'
orig_text_from_calc_indices='glinted along the paper as\nshe turned a page with a flutter of slender muscles in her arms'
orig_text_from_calc_indices == text_bw_tags=False

sed.prompt='gentle, commanding gesture to indicate silence'
text_bw_tags='lifted hand'
orig_text_from_calc_indices='lifted hand'
orig_text_from_calc_indices == text_bw_tags=True

sed.prompt='magazine being tossed onto a table with a soft thud'
text_bw_tags='tossing the magazine on the table'
orig_text_from_calc_indices='tossing the magazine on the table'
orig_text_from_calc_indices == text_bw_tags=True

sed.prompt='restless movement, fabric rustling'


## generate effects

## generate audio with timestamps

In [38]:
# text = samples.ARCH_WIKI_1[:200]
# pprint(text)

In [39]:
# text = '''\
# hello, this is the test when I am voicing 123 different phrases (some in parentheses),
# with newlines
# some unreadable characters: #!@%*&
# LooLLL123
# how is it??? going!!
# and some smiles: :))
# '''

In [40]:
from src.audio_generators import AudioGeneratorWithEffects

In [42]:
res.phrases[0]

CharacterPhrase(character='narrator', text='Inside, the crimson room bloomed with light. Tom and Miss Baker sat at either end of the long couch and she read aloud to him from the Saturday Evening Post—the words, murmurous and uninflected, running together in a soothing tune. The lamplight, bright on his boots and dull on the autumn-leaf yellow of her hair, glinted along the paper as she turned a page with a flutter of slender muscles in her arms.')

In [ ]:
data_for_tts = [
    {
        'modified_text': phrase.text,
        'stability': 0.5,
        'similarity_boost': 0.5,
        'style': 0.5,
    }
    for phrase in res.phrases
]

In [ ]:
AudioGeneratorWithEffects._generate_tts_audio()

In [16]:
params = TTSParams(
    voice_id="8opUN7sGOKbyojnjvNdl",
    text=text,
    # seed=672
)

In [17]:
params.output_format

<AudioOutputFormat.MP3_44100_192: 'mp3_44100_192'>

In [18]:
list(params)

[('voice_id', '8opUN7sGOKbyojnjvNdl'),
 ('text',
  'hello, this is the test when I am voicing 123 different phrases (some in parentheses),\nwith newlines\nsome unreadable characters: #!@%*&\nLooLLL123\nhow is it??? going!!\nand some smiles: :))\n'),
 ('output_format', <AudioOutputFormat.MP3_44100_192: 'mp3_44100_192'>),
 ('audio_model_id', Ellipsis),
 ('language_code', Ellipsis),
 ('voice_settings', Ellipsis),
 ('seed', 672),
 ('previous_text', Ellipsis),
 ('next_text', Ellipsis),
 ('previous_request_ids', Ellipsis),
 ('next_request_ids', Ellipsis)]

In [19]:
params.to_dict()

{'voice_id': '8opUN7sGOKbyojnjvNdl',
 'text': 'hello, this is the test when I am voicing 123 different phrases (some in parentheses),\nwith newlines\nsome unreadable characters: #!@%*&\nLooLLL123\nhow is it??? going!!\nand some smiles: :))\n',
 'output_format': <AudioOutputFormat.MP3_44100_192: 'mp3_44100_192'>,
 'seed': 672}

In [22]:
response = await tts.tts_w_timestamps(params)

2024-10-29 22:55:03,113 [INFO] audio-books (tts.py): request to 11labs TTS endpoint with params {'voice_id': '8opUN7sGOKbyojnjvNdl', 'output_format': <AudioOutputFormat.MP3_44100_192: 'mp3_44100_192'>, 'seed': 672} for the following text: "hello, this is the test when I am voicing 123 different phrases (some in parentheses),
with newlines
some unreadable characters: #!@%*&
LooLLL123
how is it??? going!!
and some smiles: :))
"
2024-10-29 22:55:06,333 [INFO] httpx (_client.py): HTTP Request: POST https://api.elevenlabs.io/v1/text-to-speech/8opUN7sGOKbyojnjvNdl/with-timestamps?output_format=mp3_44100_192 "HTTP/1.1 200 OK"


In [23]:
response.write_audio_to_file('tmp.672.2', params.output_format)

2024-10-29 22:55:07,034 [INFO] audio-books (utils.py): saving to: "tmp.672.2.mp3"


'tmp.672.2.mp3'

In [64]:
response_raw = await tts.ELEVEN_CLIENT_ASYNC.text_to_speech.convert_with_timestamps(
    **params.to_dict()
)

/Users/ales/dev/python-venvs/ai-audio-books/lib/python3.12/site-packages/pydantic/main.py:390: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `ellipsis` with value `Ellipsis` - serialized value may not be as expected
  Expected `str` but got `ellipsis` with value `Ellipsis` - serialized value may not be as expected
  Expected `VoiceSettings` but got `ellipsis` with value `Ellipsis` - serialized value may not be as expected
  Expected `int` but got `ellipsis` with value `Ellipsis` - serialized value may not be as expected
  Expected `str` but got `ellipsis` with value `Ellipsis` - serialized value may not be as expected
  Expected `str` but got `ellipsis` with value `Ellipsis` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
2024-10-27 21:44:22,111 [INFO] httpx (_client.py): HTTP Request: POST https://api.elevenlabs.io/v1/text-to-speech/8opUN7sGOKbyojnjvNdl/with-timestamps?output_format=mp3_44100_192 "HTTP/1.1 200 OK"


In [75]:
response_parsed = TTSTimestampsResponse.model_validate(response_raw)

In [86]:
t = response_parsed.alignment.text_joined
t

'hello, this is the test when I am voicing 123 different phrases (some in parentheses),\nwith newlines\nsome unreadable characters: #!@%*&\nLooLLL123\nhow is it??? going!!\nand some smiles: :))\n'

In [89]:
from src.schemas import TTSTimestampsAlignemnt

In [94]:
a1 = TTSTimestampsAlignemnt(
    characters=list('abc'),
    character_start_times_seconds=[0.1, 0.2, 0.3],
    character_end_times_seconds=[0.15, 0.25, 0.35],
)
a2 = TTSTimestampsAlignemnt(
    characters=list('def'),
    character_start_times_seconds=[0.1, 0.2, 0.3],
    character_end_times_seconds=[0.15, 0.25, 0.35],
)
a3 = TTSTimestampsAlignemnt(
    characters=list("ghi"),
    character_start_times_seconds=[0.1, 0.2, 0.3],
    character_end_times_seconds=[0.15, 0.25, 0.35],
)

In [108]:
a = TTSTimestampsAlignemnt.combine_alignments(alignments=[a1, a2, a3])

In [109]:
a.to_dataframe()

,char,start,end
0,a,0.10,0.15
1,b,0.20,0.25
2,c,0.30,0.35
3,d,0.45,0.50
4,e,0.55,0.60
5,f,0.65,0.70
6,g,0.80,0.85
7,h,0.90,0.95
8,i,1.00,1.05


In [110]:
a.get_start_time_by_char_ix(3)

0.44999999999999996

In [111]:
a.get_end_time_by_char_ix(6)

0.85

## compare audio quality for different formats

In [5]:
text = samples.ARCH_WIKI_1[:200]
pprint(text)

('This document is an annotated index of popular articles and important '
 'information for improving and adding functionalities to the installed Arch '
 'system. Readers are assumed to have read and followed t')


In [12]:
params_base = TTSParams(
    voice_id="8opUN7sGOKbyojnjvNdl",
    text="hello, how are you doing? this is the test aiming to decide which audio quality option to use",
    # text=text,
)

In [13]:
# out_dp = "data/compare_audio_quality2"
# os.makedirs(out_dp, exist_ok=True)

# for audio_format in AudioOutputFormat:
#     if audio_format is AudioOutputFormat.ULAW_8000:
#         continue

#     params = params_base.model_copy(deep=True)
#     params.output_format = audio_format

#     response_raw = await ELEVEN_CLIENT_ASYNC.text_to_speech.convert_with_timestamps(
#         **params.to_dict()
#     )
#     response_parsed = TTSTimestampsResponse.model_validate(response_raw)

#     filepath_no_ext = os.path.join(out_dp, f"compare.{audio_format}")
#     out_fp = response_parsed.write_audio_to_file(
#         filepath_no_ext=filepath_no_ext, audio_format=audio_format
#     )

/Users/ales/dev/python-venvs/ai-audio-books/lib/python3.12/site-packages/pydantic/main.py:390: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `ellipsis` with value `Ellipsis` - serialized value may not be as expected
  Expected `str` but got `ellipsis` with value `Ellipsis` - serialized value may not be as expected
  Expected `VoiceSettings` but got `ellipsis` with value `Ellipsis` - serialized value may not be as expected
  Expected `int` but got `ellipsis` with value `Ellipsis` - serialized value may not be as expected
  Expected `str` but got `ellipsis` with value `Ellipsis` - serialized value may not be as expected
  Expected `str` but got `ellipsis` with value `Ellipsis` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
2024-10-27 18:16:54,779 [INFO] httpx (_client.py): HTTP Request: POST https://api.elevenlabs.io/v1/text-to-speech/8opUN7sGOKbyojnjvNdl/with-timestamps?output_format=mp3_22050_32 "HTTP/1.1 200 OK"
2024-10

In [11]:
!ffprobe data/compare_audio_quality/compare.mp3_44100_64.mp3

ffprobe version 7.0.1 Copyright (c) 2007-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex -